# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [85]:
import pandas as pd

# Carregar a base de dados
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Contar o número total de registros
total_registros = sinasc.shape[0]
print(f"Número total de registros: {total_registros}")

# Remover duplicatas
sinasc.drop_duplicates(inplace=True)

# Contar o número de registros após remover duplicatas
registros_nao_duplicados = sinasc.shape[0]
print(f"Número de registros após remover duplicatas: {registros_nao_duplicados}")

# Verificar se havia linhas duplicadas
if total_registros == registros_nao_duplicados:
    print("Não havia linhas duplicadas na base de dados.")
else:
    print(f"Haviam {total_registros - registros_nao_duplicados} linhas duplicadas na base de dados.")

Número total de registros: 27028
Número de registros após remover duplicatas: 27028
Não havia linhas duplicadas na base de dados.


In [86]:
# Contar o número de valores missing por variável
missing_values = sinasc.isnull().sum()
print(missing_values)

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [87]:
# Criar uma lista com as colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

# Selecionar apenas as colunas de interesse no DataFrame
sinasc_selecao = sinasc[colunas_interesse]

# Imprimir o novo DataFrame
print(sinasc_selecao)

       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 semanas    Única          4     9.0  
2      37 a 41 semanas  

In [88]:
# Remover os registros onde 'APGAR5' é não preenchido
sinasc_selecao = sinasc_selecao.dropna(subset=['APGAR5'])

# Contar o número de linhas após a remoção
num_linhas = sinasc_selecao.shape[0]
print(f"Número de linhas após a remoção de 'APGAR5' não preenchido: {num_linhas}")

# Contar novamente o número de valores missing para cada variável
num_missings = sinasc_selecao.isnull().sum()
print("\nNúmero de valores missing para cada variável após a remoção:")
print(num_missings)

Número de linhas após a remoção de 'APGAR5' não preenchido: 26925

Número de valores missing para cada variável após a remoção:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [89]:
# Preenche os valores faltantes (representados como NaN) com o valor 9
sinasc_selecao['ESTCIVMAE'] = sinasc_selecao['ESTCIVMAE'].fillna(9)
sinasc_selecao['CONSULTAS'] = sinasc_selecao['CONSULTAS'].fillna(9)
sinasc_selecao

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [90]:
# Substituir os valores faltantes na coluna 'QTDFILVIVO' por zero
sinasc_selecao['QTDFILVIVO'] = sinasc_selecao['QTDFILVIVO'].fillna(0)

In [91]:
# Substituir valores faltantes pela mediana
sinasc_selecao['IDADEMAE'].fillna(sinasc_selecao['IDADEMAE'].median(), inplace=True)

In [92]:
# Substituir valores faltantes pelo valor mais frequente
estcivmae_mais_frequente = sinasc_selecao['ESTCIVMAE'].mode()[0]
sinasc_selecao['ESTCIVMAE'].fillna(estcivmae_mais_frequente, inplace=True)

In [93]:
# Substituir valores faltantes pelo valor mais frequente
escmae_mais_frequente = sinasc_selecao['ESCMAE'].mode()[0]
sinasc_selecao['ESCMAE'].fillna(escmae_mais_frequente, inplace=True)

In [94]:
# Consultar e substituir valores faltantes pela mediana
gestacao_mais_frequente = sinasc_selecao['GESTACAO'].mode()[0]
sinasc_selecao['GESTACAO'].fillna(gestacao_mais_frequente, inplace=True)

In [95]:
# Substituir valores faltantes pelo valor mais frequente
gravidez_mais_frequente = sinasc_selecao['GRAVIDEZ'].mode()[0]
sinasc_selecao['GRAVIDEZ'].fillna(gravidez_mais_frequente, inplace=True)

In [96]:
# Substituir valores faltantes pela mediana
sinasc_selecao['CONSULTAS'].fillna(sinasc_selecao['CONSULTAS'].median(), inplace=True)

In [97]:
# Verificar se ainda há valores faltantes
sinasc_selecao.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [98]:
# Definir os limites das categorias
bins = [-1, 3, 5, 7, 10]

# Definir os rótulos das categorias
labels = ['Asfixia Severa', 'Asfixia Moderada', 'Asfixia Leve', 'Normal']

# Criar a nova coluna 'APGAR5_CAT'
sinasc_selecao['APGAR5_CAT'] = pd.cut(sinasc_selecao['APGAR5'], bins=bins, labels=labels)

# Calcular as frequências dessa categorização
frequencias = sinasc_selecao['APGAR5_CAT'].value_counts()
print(frequencias)

Normal              26463
Asfixia Leve          320
Asfixia Severa         74
Asfixia Moderada       68
Name: APGAR5_CAT, dtype: int64


In [99]:
# Renomear as colunas para snake case
sinasc_selecao.columns = sinasc_selecao.columns.str.lower().str.replace(' ', '_')

# Verificar o resultado
print(sinasc_selecao.columns)

Index(['locnasc', 'idademae', 'estcivmae', 'escmae', 'qtdfilvivo', 'gestacao',
       'gravidez', 'consultas', 'apgar5', 'apgar5_cat'],
      dtype='object')
